In [1]:
'''We have used coopersmith function to calcualte the rules ,the below coopersmith function is referred from https://github.com/mimoo/RSA-and-LLL-attacks/blob/master/coppersmith.sage to implement the coppersmith function'''

'We have used coopersmith function to calcualte the rules ,the below coopersmith function is referred from https://github.com/mimoo/RSA-and-LLL-attacks/blob/master/coppersmith.sage to implement the coppersmith function'

In [2]:
def coppersmith_howgrave_univariate(pol, modulus, beta, mm, tt, XX):
    """
    Coppersmith revisited by Howgrave-Graham
    
    finds a solution if:
    * b|modulus, b >= modulus^beta , 0 < beta <= 1
    * |x| < XX
    """
    #
    # init
    #
    dd = pol.degree()
    nn = dd * mm + tt

    #
    # checks
    #
    if not 0 < beta <= 1:
        raise ValueError("beta should belongs in (0, 1]")

    if not pol.is_monic():
        raise ArithmeticError("Polynomial must be monic.")

    #
    # calculate bounds and display them
    #
    """
    * we want to find g(x) such that ||g(xX)|| <= b^m / sqrt(n)
    * we know LLL will give us a short vector v such that:
    ||v|| <= 2^((n - 1)/4) * det(L)^(1/n)
    * we will use that vector as a coefficient vector for our g(x)
    
    * so we want to satisfy:
    2^((n - 1)/4) * det(L)^(1/n) < N^(beta*m) / sqrt(n)
    
    so we can obtain ||v|| < N^(beta*m) / sqrt(n) <= b^m / sqrt(n)
    (it's important to use N because we might not know b)
    """
#     if debug:
#         # t optimized?
#         print("\n# Optimized t?\n")
#         print("we want X^(n-1) < N^(beta*m) so that each vector is helpful")
#         cond1 = RR(XX^(nn-1))
#         print("* X^(n-1) = ", cond1)
#         cond2 = pow(modulus, beta*mm)
#         print("* N^(beta*m) = ", cond2)
#         print("* X^(n-1) < N^(beta*m) \n-> GOOD" if cond1 < cond2 else "* X^(n-1) >= N^(beta*m) \n-> NOT GOOD")
        
#         # bound for X
#         print("\n# X bound respected?\n")
#         print("we want X <= N^(((2*beta*m)/(n-1)) - ((delta*m*(m+1))/(n*(n-1)))) / 2 = M")
#         print("* X =", XX)
#         cond2 = RR(modulus^(((2*beta*mm)/(nn-1)) - ((dd*mm*(mm+1))/(nn*(nn-1)))) / 2)
#         print("* M =", cond2)
#         print("* X <= M \n-> GOOD" if XX <= cond2 else "* X > M \n-> NOT GOOD")

#         # solution possible?
#         print("\n# Solutions possible?\n")
#         detL = RR(modulus^(dd * mm * (mm + 1) / 2) * XX^(nn * (nn - 1) / 2))
#         print("we can find a solution if 2^((n - 1)/4) * det(L)^(1/n) < N^(beta*m) / sqrt(n)")
#         cond1 = RR(2^((nn - 1)/4) * detL^(1/nn))
#         print("* 2^((n - 1)/4) * det(L)^(1/n) = ", cond1)
#         cond2 = RR(modulus^(beta*mm) / sqrt(nn))
#         print("* N^(beta*m) / sqrt(n) = ", cond2)
#         print("* 2^((n - 1)/4) * det(L)^(1/n) < N^(beta*m) / sqrt(n) \n-> SOLUTION WILL BE FOUND" if cond1 < cond2 else "* 2^((n - 1)/4) * det(L)^(1/n) >= N^(beta*m) / sqroot(n) \n-> NO SOLUTIONS MIGHT BE FOUND (but we never know)")

#         # warning about X
#         print("\n# Note that no solutions will be found _for sure_ if you don't respect:\n* |root| < X \n* b >= modulus^beta\n")
    
    #
    # Coppersmith revisited algo for univariate
    #

    # change ring of pol and x
    polZ = pol.change_ring(ZZ)
    x = polZ.parent().gen()

    # compute polynomials
    gg = []
    for ii in range(mm):
        for jj in range(dd):
            gg.append((x * XX)**jj * modulus**(mm - ii) * polZ(x * XX)**ii)
    for ii in range(tt):
        gg.append((x * XX)**ii * polZ(x * XX)**mm)
    
    # construct lattice B
    BB = Matrix(ZZ, nn)

    for ii in range(nn):
        for jj in range(ii+1):
            BB[ii, jj] = gg[ii][jj]

    # display basis matrix
#     if debug:
#         matrix_overview(BB, modulus^mm)

    # LLL
    BB = BB.LLL()

    # transform shortest vector in polynomial    
    new_pol = 0
    for ii in range(nn):
        new_pol += x**ii * BB[0, ii] / XX**ii

    # factor polynomial
    potential_roots = new_pol.roots()
    print("potential roots:", potential_roots)

    # test roots
    roots = []
    for root in potential_roots:
        if root[0].is_integer():
            result = polZ(ZZ(root[0]))
            if gcd(modulus, result) >= modulus^beta:
                roots.append(ZZ(root[0]))

    # 
    return roots

In [3]:
padding_text = "Goldfish: This door has RSA encryption with exponent 5 and the password is "

In [15]:
# Converted the padding text into its corresponding Ascii value.
padding_integer=[]
for i in padding_text:
    padding_integer.append(ord(i))
padding_integer

[71,
 111,
 108,
 100,
 102,
 105,
 115,
 104,
 58,
 32,
 84,
 104,
 105,
 115,
 32,
 100,
 111,
 111,
 114,
 32,
 104,
 97,
 115,
 32,
 82,
 83,
 65,
 32,
 101,
 110,
 99,
 114,
 121,
 112,
 116,
 105,
 111,
 110,
 32,
 119,
 105,
 116,
 104,
 32,
 101,
 120,
 112,
 111,
 110,
 101,
 110,
 116,
 32,
 53,
 32,
 97,
 110,
 100,
 32,
 116,
 104,
 101,
 32,
 112,
 97,
 115,
 115,
 119,
 111,
 114,
 100,
 32,
 105,
 115,
 32]

In [19]:
# Combined the above padding value integers after converting them into the binary of 8 bits each.
final_padding = ''
for i in padding_integer:
    final_padding += f'{i:08b}'
    
final_padding

'010001110110111101101100011001000110011001101001011100110110100000111010001000000101010001101000011010010111001100100000011001000110111101101111011100100010000001101000011000010111001100100000010100100101001101000001001000000110010101101110011000110111001001111001011100000111010001101001011011110110111000100000011101110110100101110100011010000010000001100101011110000111000001101111011011100110010101101110011101000010000000110101001000000110000101101110011001000010000001110100011010000110010100100000011100000110000101110011011100110111011101101111011100100110010000100000011010010111001100100000'

In [22]:
# given exponent =5 
e = 5
# cipher C
C = 60278917802604119062164317348887608230355852573467939559623685511662664827094799119961975647948079028381561909992682799715331816075454441008141268270670616332094929226033013419888728755237642448079785343995296657167212939406356487301422599540737091376110873706828441276659546878209779201560390761048657774255

N = 84364443735725034864402554533826279174703893439763343343863260342756678609216895093779263028809246505955647572176682669445270008816481771701417554768871285020442403001649254405058303439906229201909599348669565697534331652019516409514800265887388539283381053937433496994442146419682027649079704982600857517093
# Ring of integers modulo
Zn = IntegerModRing(N)

In [23]:
Zn

Ring of integers modulo 84364443735725034864402554533826279174703893439763343343863260342756678609216895093779263028809246505955647572176682669445270008816481771701417554768871285020442403001649254405058303439906229201909599348669565697534331652019516409514800265887388539283381053937433496994442146419682027649079704982600857517093

In [7]:
'''Since these variables were required to implement the algoritm So are refere
Ref:https://github.com/mimoo/RSA-and-LLL-attacks/'''
beta = 1                                # b = N
epsilon = beta / 7 

In [24]:
# the code will find out the roots and will print those it will run for 201 times untill the roots are found and ones found it will break.
for leng in range(0, 201, 4):

    #the below snippets are reffered from : https://github.com/mimoo/RSA-and-LLL-attacks/blob/master/coppersmith.sage
    P.<M> = PolynomialRing(Zn)
    pol = ((int(final_padding, 2)<<leng) + M)^e - C   
    dd = pol.degree()
    mm = ceil(beta**2 / (dd * epsilon))     # optimized value
    tt = floor(dd * mm * ((1/beta) - 1))    # optimized value
    XX = ceil(N**((beta**2/dd) - epsilon))  # optimized value
    roots = coppersmith_howgrave_univariate(pol, N, beta, mm, tt, XX)
    
    if not roots:
        continue
    else:
        print("The correct root is :", bin(roots[0])[2:])
        break

potential roots: []
potential roots: []
potential roots: []


potential roots: []
potential roots: []
potential roots: []


potential roots: []
potential roots: []
potential roots: []


potential roots: []
potential roots: []
potential roots: []


potential roots: []
potential roots: []
potential roots: []


potential roots: []
potential roots: []
potential roots: []


potential roots: []
potential roots: []
potential roots: [(317438007725212639901273, 1)]
The correct root is : 1000011001110000101100101010000001101110110111101001100011011110011011001011001


In [25]:
plain = bin(roots[0])[2:]

In [26]:
type(plain)

<class 'str'>

In [27]:
len(plain)
plain

'1000011001110000101100101010000001101110110111101001100011011110011011001011001'

In [28]:
#Since the length is not a multiple of 8 added a zero to the plain
plain = '0'+plain

In [29]:
print("final plain:")
plain

final plain:


'01000011001110000101100101010000001101110110111101001100011011110011011001011001'

In [34]:
# converting back the plain back to the text and finally the password is extracted.
res = ''

for i in range(0,len(plain),8):
    s = plain[i:i+8]
    n = int(s,2)
    ch = chr(n)
    res += ch

In [35]:
print("The Password is :"+res)

The Password is :C8YP7oLo6Y
